In [9]:
import pandas as pd

# Get NASDAQ-100 tickers & sectors from Wikipedia
nasdaq100_url = "https://en.wikipedia.org/wiki/NASDAQ-100"
nasdaq100_table = pd.read_html(nasdaq100_url, header=0)[4]  # Extract table (4th table is NASDAQ-100 list)

# Keep only relevant columns
nasdaq100_data = nasdaq100_table[['Symbol', 'Company', 'GICS Sector']]
nasdaq100_data.columns = ['Ticker', 'Company', 'Sector']

# Display first few rows
print(nasdaq100_data.head())

# Save to CSV (Optional)
nasdaq100_data.to_csv("nasdaq100_tickers.csv", index=False)


  Ticker                  Company                  Sector
0   ADBE               Adobe Inc.  Information Technology
1    AMD   Advanced Micro Devices  Information Technology
2   ABNB                   Airbnb  Consumer Discretionary
3  GOOGL  Alphabet Inc. (Class A)  Communication Services
4   GOOG  Alphabet Inc. (Class C)  Communication Services


In [ ]:
import yfinance as yf

# Load NASDAQ-100 tickers
tickers = nasdaq100_data["Ticker"].tolist()

# Define time period
start_date = "2005-01-01"
end_date = "2025-01-01"

# Create dictionary to store stock data
stock_data = pd.DataFrame()
OCHLV = ["Open","Close","High","Low","Volume"]
# Fetch data for each stock
for ticker in tickers:
    try:
        df_ = yf.download(ticker, start=start_date, end=end_date, progress=False)
        temp = df_.reset_index().reset_index()
        stock_df = pd.DataFrame(temp["Date"])
        
        for u in OCHLV:
            t = temp[u][ticker]
            tt = pd.DataFrame({u:t})
            stock_df = pd.concat([stock_df,tt],axis = 1)

        stock_df["Ticker"] = ticker  
        stock_data = pd.concat([stock_data,stock_df])
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

stock_data.to_csv("nasdaq100_stock_prices.csv")


In [110]:
## timestamp 
price_df = pd.read_csv("nasdaq100_stock_prices.csv")
price_df = price_df.merge(nasdaq100_data, on = "Ticker", how = "left")
price_df = price_df.drop(price_df.columns[0], axis=1)
price_df['Date'] = pd.to_datetime(price_df['Date'])
price_df.head()

,Date,Open,Close,High,Low,Volume,Ticker,Company,Sector
0,2005-01-03,31.493821,30.838949,31.743772,30.768963,5508800,ADBE,Adobe Inc.,Information Technology
1,2005-01-04,31.098899,30.024111,31.183883,29.674179,7515400,ADBE,Adobe Inc.,Information Technology
2,2005-01-05,30.084099,29.859142,30.444029,29.859142,3566600,ADBE,Adobe Inc.,Information Technology
3,2005-01-06,30.109092,29.364239,30.164081,29.239263,6159600,ADBE,Adobe Inc.,Information Technology
4,2005-01-07,29.499211,29.384233,29.749162,28.844340,8512400,ADBE,Adobe Inc.,Information Technology


In [147]:
# rolling avg dollar volume 
price_df['dollar_vol'] = price_df[['Close','Volume']].prod(axis = 1)
price_df['dollar_vol_1m'] = (
    price_df.groupby('Ticker')['dollar_vol']
    .rolling(window=21, min_periods=1)  
    .mean()
    .reset_index(level=0, drop=True)  
)
price_df['dollar_vol_rank'] = (price_df.groupby(price_df['Date'])
                             .dollar_vol_1m
                             .rank(ascending=False))

In [148]:
price_df

,Date,Open,Close,High,Low,Volume,Ticker,Company,Sector,dollar_vol,dollar_vol_1m,dollar_vol_rank
0,2005-01-03,31.493821,30.838949,31.743772,30.768963,5508800,ADBE,Adobe Inc.,Information Technology,1.698856e+08,1.698856e+08,18.0
1,2005-01-04,31.098899,30.024111,31.183883,29.674179,7515400,ADBE,Adobe Inc.,Information Technology,2.256432e+08,1.977644e+08,17.0
2,2005-01-05,30.084099,29.859142,30.444029,29.859142,3566600,ADBE,Adobe Inc.,Information Technology,1.064956e+08,1.673415e+08,18.0
3,2005-01-06,30.109092,29.364239,30.164081,29.239263,6159600,ADBE,Adobe Inc.,Information Technology,1.808720e+08,1.707241e+08,18.0
4,2005-01-07,29.499211,29.384233,29.749162,28.844340,8512400,ADBE,Adobe Inc.,Information Technology,2.501303e+08,1.866053e+08,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...
425842,2024-12-24,185.979996,187.259995,187.589996,184.679993,654200,ZS,Zscaler,Information Technology,1.225055e+08,4.944919e+08,63.0
425843,2024-12-26,186.000000,187.630005,188.500000,185.869995,806500,ZS,Zscaler,Information Technology,1.513236e+08,4.840806e+08,62.0
425844,2024-12-27,185.259995,184.559998,185.929993,181.259995,1256100,ZS,Zscaler,Information Technology,2.318258e+08,4.823277e+08,61.0
425845,2024-12-30,181.529999,183.130005,184.404999,180.100006,1032400,ZS,Zscaler,Information Technology,1.890634e+08,4.761139e+08,61.0


In [143]:
!pip install numpy --upgrade

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.2.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.3-cp312-cp312-win_amd64.whl (12.6 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.3 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 2.2.3 which is incompatible.


In [129]:
price_df[price_df['Ticker'] == 'GOOGL']

,Date,Open,Close,High,Low,Volume,Ticker,Company,Sector,dollar_vol,dollar_vol_1m
11086,2005-01-03,4.922097,5.054500,5.077689,4.873723,633134232,GOOGL,Alphabet Inc. (Class A),Communication Services,3.200177e+09,3.200177e+09
11087,2005-01-04,5.021835,4.849786,5.059985,4.824353,549685764,GOOGL,Alphabet Inc. (Class A),Communication Services,2.665858e+09,2.933017e+09
11088,2005-01-05,4.823605,4.825102,4.909629,4.793186,329134536,GOOGL,Alphabet Inc. (Class A),Communication Services,1.588108e+09,2.484714e+09
11089,2005-01-06,4.864249,4.701425,4.884694,4.680730,415068516,GOOGL,Alphabet Inc. (Class A),Communication Services,1.951414e+09,2.351389e+09
11090,2005-01-07,4.753539,4.833579,4.843553,4.707160,386129484,GOOGL,Alphabet Inc. (Class A),Communication Services,1.866387e+09,2.254389e+09
...,...,...,...,...,...,...,...,...,...,...,...
16114,2024-12-24,194.839996,196.110001,196.110001,193.779999,10403300,GOOGL,Alphabet Inc. (Class A),Communication Services,2.040191e+09,5.897950e+09
16115,2024-12-26,195.149994,195.600006,196.750000,194.380005,12046600,GOOGL,Alphabet Inc. (Class A),Communication Services,2.356315e+09,5.745928e+09
16116,2024-12-27,194.949997,192.759995,195.320007,190.649994,18891400,GOOGL,Alphabet Inc. (Class A),Communication Services,3.641506e+09,5.754536e+09
16117,2024-12-30,189.800003,191.240005,192.550003,189.119995,14264700,GOOGL,Alphabet Inc. (Class A),Communication Services,2.727981e+09,5.729357e+09
